# Sanity Check: Local Inference

You need to generate predictions so you can test the model.  Axolotl uploaded the trained model to

[parlance-labs/hc-mistral-alpaca](https://huggingface.co/parlance-labs/hc-mistral-alpaca)

In [1]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
# model_id='parlance-labs/hc-mistral-alpaca' # this will be different for you based upon hub_model_id
model_id='./qlora-alpaca-out' # this will be different for you based upon hub_model_id
model = AutoPeftModelForCausalLM.from_pretrained(model_id).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

### Prompt Template

Next, we have to construct a prompt template that is as close as possible to the prompt template we saw earlier.

In [3]:
def prompt(nlq, cols):
    return f"""Honeycomb is an observability platform that allows you to write queries to inspect trace data. You are an assistant that takes a natural language query (NLQ) and a list of valid columns and produce a Honeycomb query.

### Instruction:

NLQ: "{nlq}"

Columns: {cols}

### Response:
"""

def prompt_tok(nlq, cols, return_ids=False):
    _p = prompt(nlq, cols)
    input_ids = tokenizer(_p, return_tensors="pt", truncation=True).input_ids.cuda()
    out_ids = model.generate(input_ids=input_ids, max_new_tokens=5000, 
                          do_sample=False)
    ids = out_ids.detach().cpu().numpy()
    if return_ids: return out_ids
    return tokenizer.batch_decode(ids, 
                                  skip_special_tokens=True)[0][len(_p):]

## Sanity Check Examples

Next, we sanity check a few examples to make sure that:

- Our prompt template is constructed correctly (indeed I made some mistakes at first!)
- The model is working as expected.

In [4]:
nlq = "Exception count by exception and caller"
cols = ['error', 'exception.message', 'exception.type', 'exception.stacktrace', 'SampleRate', 'name', 'db.user', 'type', 'duration_ms', 'db.name', 'service.name', 'http.method', 'db.system', 'status_code', 'db.operation', 'library.name', 'process.pid', 'net.transport', 'messaging.system', 'rpc.system', 'http.target', 'db.statement', 'library.version', 'status_message', 'parent_name', 'aws.region', 'process.command', 'rpc.method', 'span.kind', 'serializer.name', 'net.peer.name', 'rpc.service', 'http.scheme', 'process.runtime.name', 'serializer.format', 'serializer.renderer', 'net.peer.port', 'process.runtime.version', 'http.status_code', 'telemetry.sdk.language', 'trace.parent_id', 'process.runtime.description', 'span.num_events', 'messaging.destination', 'net.peer.ip', 'trace.trace_id', 'telemetry.instrumentation_library', 'trace.span_id', 'span.num_links', 'meta.signal_type', 'http.route']

nlq2 = "ssv-customer-identification"
cols2 = ['ssv_debit_card_fast_processing_sli', 'ssv_debit_card_fast_processing_sli_exclude_warmup', 'fincrime_ems_success_sli', 'event_timestamp', 'fincrime_ems_sync_sla_upheld_rate', 'fincrime_ems_sync_sla_missed_rate_sli', 'fincrime_ems_success', 'interbank_bacs_debit_payments_successfully_processed_on_processing_date', 'interbank_bacs_credit_payments_successfully_processed_on_processing_date', 'date_iso', 'CustomerIdentificationExists', 'interbank_saga_processes_success_rate', 'fincrime_ems_sync_sla_missed_rate', 'fincrime_ems_success_rate', 'interbank_inbound_fps_payment_latency', 'sagaStatusExists', 'timestamp.is_nine_to_five', 'timestamp.event_hour', 'did_skitty_onboarding_fail', 'timestamp.is_weekend', 'onfido.api.url', 'is_weekend', 'isPending', 'was_skitty_onboarding_successful', 'trace.parent_id', 'duration_ms', 'http.status_code', 'name', 'service.name', 'exception.message', 'parent_name', 'db.statement', 'error', 'http.route']

In [5]:
out = prompt_tok(nlq, cols)
print(nlq, '\n', out.strip())

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Exception count by exception and caller 
 "{'breakdowns': ['exception.message', 'exception.type'], 'calculations': [{'op': 'COUNT'}], 'filters': [{'column': 'exception.message', 'op': 'exists'}, {'column': 'exception.type', 'op': 'exists'}], 'orders': [{'op': 'COUNT', 'order': 'descending'}], 'time_range': 7200}"


In [6]:
out = prompt_tok(nlq2, cols2)
print(nlq, '\n', out.strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Exception count by exception and caller 
 "{'breakdowns': ['ssv_debit_card_fast_processing_sli', 'ssv_debit_card_fast_processing_sli_exclude_warmup'], 'calculations': [{'op': 'COUNT'}], 'filters': [{'column': 'ssv_debit_card_fast_processing_sli', 'op': 'exists'}, {'column': 'ssv_debit_card_fast_processing_sli_exclude_warmup', 'op': 'exists'}], 'orders': [{'op': 'COUNT', 'order': 'descending'}], 'time_range': 7200}"


## Homework - Token Check

Optional Homework!  Check that there aren't any token issues https://hamel.dev/notes/llm/finetuning/05_tokenizer_gotchas.html